In [1]:
import timm

import numpy as np
import torch.nn as nn
import torch
from torchvision import datasets, models, transforms

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import gc

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResBlock(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(ResBlock, self).__init__()
        self.downsample = out_channels//in_channels
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=self.downsample, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=self.downsample)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)

        if self.downsample > 1:
            residual = self.shortcut(x)

        out += residual
        out = self.relu(out)

        return out
    
class ResNet(nn.Module):

    def __init__(self, in_channels, nblocks, fmaps):
        super(ResNet, self).__init__()
        self.fmaps = fmaps
        self.nblocks = nblocks
        
        self.conv0 = nn.Conv2d(in_channels, fmaps[0], kernel_size=7, stride=2, padding=1)
        self.layer1 = self.block_layers(self.nblocks, [fmaps[0],fmaps[0]])
        self.layer2 = self.block_layers(1, [fmaps[0],fmaps[1]])
        self.layer3 = self.block_layers(self.nblocks, [fmaps[1],fmaps[1]])
        self.fc = nn.Linear(fmaps[1], 1)
        #self.FCN = nn.Sequential(
        #        nn.Linear(fmaps[1], 128), nn.ReLU(), nn.Dropout(p=0.2),
        #        nn.Linear(128, 128), nn.ReLU(), nn.Dropout(p=0.2),
        #        nn.Linear(128, 128), nn.ReLU(), nn.Dropout(p=0.2),
        #        nn.Linear(128, 3)
        #        )
        
    def block_layers(self, nblocks, fmaps):
        layers = []
        for _ in range(nblocks):
            layers.append(ResBlock(fmaps[0], fmaps[1]))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv0(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = F.max_pool2d(x, kernel_size=x.size()[2:])
        x = x.view(x.size()[0], self.fmaps[1])
        x = self.fc(x)
        #x = self.FCN(x)
        x = x.squeeze()
        return x

In [3]:
def metric(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)
    return auc

def straightner(a):
    A = np.zeros((a[0].shape[0]*len(a)))
    start_index = 0
    end_index = 0
    for i in range(len(a)):
        start_index = i*a[0].shape[0]
        end_index = start_index+a[0].shape[0]
        A[start_index:end_index] = a[i]
    return A

def predictor(outputs):
    return np.argmax(outputs, axis = 1)

In [6]:
model = ResNet(3, 3, [16, 32])

train_transform = transforms.Compose([transforms.Resize((125,125)),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.RandomRotation(20),
                            transforms.ToTensor()
                           ])
test_transform = transforms.Compose([transforms.Resize((125,125)),
                            transforms.ToTensor()
                           ])


dataset_Train = datasets.ImageFolder('./Quark_Gluon_Mini/Train/', transform=train_transform)
dataset_Test = datasets.ImageFolder('./Quark_Gluon_Mini/Test/', transform =test_transform)
dataloader_train = torch.utils.data.DataLoader(dataset_Train, batch_size=128, shuffle=True, drop_last = True, num_workers=0, pin_memory = False)
dataloader_test = torch.utils.data.DataLoader(dataset_Test, batch_size=128, shuffle=True, drop_last = True, num_workers=0, pin_memory = False)    


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr = 5.e-4)
criterion = nn.BCEWithLogitsLoss()
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,20], gamma=0.5)

model = model.to("cuda")


In [4]:
import wandb
wandb.login(key="cb53927c12bd57a0d943d2dedf7881cfcdcc8f09")
wandb.init(
    project = "Trash_",
    name = "Literature_code"
)

scaler = torch.cuda.amp.GradScaler()
#--------------------------
wandb.watch(model, log_freq=50)
#---------------------------
w_intr = 1000

for epoch in range(30):
    train_loss = 0
    val_loss = 0
    train_steps = 0
    test_steps = 0
    label_list = []
    outputs_list = []
    train_auc = 0
    test_auc = 0
    model.train()
    for image, label in tqdm(dataloader_train):
        image = image.to("cuda")
        label = label.to("cuda")
        #optimizer.zero_grad()
        for param in model.parameters():
            param.grad = None

        outputs = model(image[:,0,:,:].unsqueeze(1))
        loss = criterion(outputs, label.float())
        label_list.append(label.detach().cpu().numpy())
        outputs_list.append(outputs.detach().cpu().numpy())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_steps += 1
        if train_steps%w_intr == 0:
             wandb.log({"loss": loss.item()})
    with torch.no_grad():
        label_list = straightner(label_list)
        outputs_list = straightner(outputs_list)
        train_auc = metric(label_list, outputs_list) 




    #-------------------------------------------------------------------
    model.eval()
    label_list = []
    outputs_list = []
    with torch.no_grad():
        for image, label in tqdm(dataloader_test):
            image = image.to("cuda")
            label = label.to("cuda")
            outputs = model(image[:,0,:,:].unsqueeze(1))
            loss = criterion(outputs, label.float())
            label_list.append(label.detach().cpu().numpy())
            outputs_list.append(outputs.detach().cpu().numpy())
            val_loss += loss.item()
            test_steps +=1
            if test_steps%w_intr == 0:
             wandb.log({"val_loss": loss.item()})
        label_list = straightner(label_list)
        outputs_list = straightner(outputs_list)
        test_auc = metric(label_list, outputs_list)

    train_loss = train_loss/train_steps
    val_loss = val_loss/ test_steps
#     hist_loss_train.append(train_loss)
#     hist_loss_test.append(val_loss)
#     hist_auc_train.append(train_auc)
#     hist_auc_test.append(test_auc)

    print("----------------------------------------------------")
    print("Epoch No" , epoch)
    print("The Training loss of the epoch, ",train_loss)
    print("The Training AUC of the epoch,  %.3f"%train_auc)
    print("The validation loss of the epoch, ",val_loss)
    print("The validation AUC of the epoch, %.3f"%test_auc)
    print("----------------------------------------------------")
#     PATH = "model.pt"
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'scheduler': scheduler.state_dict()
#             }, PATH)
    lr_scheduler.step()
    curr_lr = lr_scheduler._last_lr[0]
    wandb.log({"Train_auc_epoch": train_auc,
              "Epoch": epoch,
              "Val_auc_epoch": test_auc,
              "Train_loss_epoch": train_loss,
              "Val_loss_epoch": val_loss,
              "Lr": curr_lr}
             )
    gc.collect()
wandb.finish()

In [152]:
images, labels = next(iter(dataloader_train))
images = images.to("cuda")
labels = labels.to("cuda")
output = model(images)

In [153]:
loss = criterion(output, labels.float())

In [25]:
loss.backward()

In [154]:
loss

tensor(27311.4648, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [151]:
optimizer.step()

In [156]:
output

tensor([-37053.7891, -58470.0234, -46584.9219, -42151.0859, -39857.3477,
        -58215.3203, -45163.8242, -65805.0703, -49551.3594, -35920.8008,
        -53223.2891, -59897.6484, -56235.8086, -40992.0391, -44947.9336,
        -43326.3477, -76288.3125, -50752.7812, -57017.9102, -38822.5195,
        -39978.5156, -52022.8164, -36821.5859, -49700.0195, -43267.5977,
        -49274.7461, -53484.2461, -40105.6016, -59079.1875, -53305.0000,
        -50406.0547, -50172.9844, -97722.7500, -38522.1992, -54185.2578,
        -42274.0547, -44518.0391, -53565.1328, -37947.0586, -37659.1367,
        -68229.4844, -52011.0312, -44492.3359, -67162.2109, -44757.2773,
        -40536.3203, -59534.0469, -49588.8047, -40495.5547, -52991.8320,
        -64703.0625, -43687.5195, -47974.8828, -35663.2773, -38176.5039,
        -33539.9023, -38790.5469, -53741.0391, -42099.4062, -67782.8828,
        -46462.1484, -52750.8906, -35765.3867, -60213.2031, -49455.6289,
        -51277.3477, -50426.6602, -60714.0078, -409